# The fundarmental group of $\Sigma_{g,1}$ 

In [1]:
require './freegroup.rb'

true

## generators

In [2]:
genus = 3
alphabet = %w(a b c d e f)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show)

a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
p a.map(&:show)
nil

["a", "b", "c", "d", "e", "f"]
["1", "a", "c", "e"]


In [3]:
comms = []
(Gens.size/2 + 1).times do |i|
  comms << Group.commutator(a[i], b[i])
end
p comms.map(&:show)

["1111", "abAB", "cdCD", "efEF"]


["1111", "abAB", "cdCD", "efEF"]

In [4]:
Rseq = []
(Gens.size/2).times do |i|
  Rseq += [a[i+1], b[i+1].inverse, a[i+1].inverse, b[i+1]]
end
Rseq.freeze
p Rseq.map(&:show) #.index('A')
nil

<main>:2: warning: already initialized constant Rseq
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here


["a", "B", "A", "b", "c", "D", "C", "d", "e", "F", "E", "f"]


# Wcode Class

In [5]:
class Wcode < Hash
  def initialize(word = Group::Identity)
    @word = word
    #---
    Rseq.each{|x| self.store(x.char, [])}
    if word != Group::Identity
      word.flatten.factors.each_with_index do |c, i|
        k = i
        self[c.char].insert(-1, 2*k)
        self[c.inverse.char].insert(0,2*k+1)
      end
    end
  end
  attr_reader :word
  #---
  def show(type=0)
    case type
    when 0
      self.map{|k, arr| arr.join(',')}.join('|')
    when 1
      self.map{|k, arr| "#{k}:" + arr.join(',')}.join('|')
    when 2
      self.each_slice(4).map{|s| s.map{|a| "#{a[1].join(',')}"}.join('|')}.join(' : ')
    when 3
      self.each_slice(4).map{|s| "(#{s.map{|a| "#{a[1].join(',')}"}.join('|')})"}.join
    end
  end
  def to_a() self.values.flatten end
  def size() self.to_a.size end
  def copy()
    wc = self.class.new
    wc.each_key{|key| wc[key] = self[key].dup}
    wc
  end
  #---
  def narrow()
    wc = self.copy
    sorted = wc.values.flatten.sort
    wc.each_key do |key|
      wc[key].map!{|v| sorted.index(v)}
    end
    wc
  end
  #---
  def to_word()
    chars = (0..self.size/2).to_a.map{|i| self.map{|key, arr| key if arr.include?(2*i)}.join}
    Word.new(chars.join)
  end
  #---
  def product(another)
    wc = self.copy
    size = self.size
    wc.map do |k, v|
      another[k].each do |n|
        n = n + self.size
        (n.odd?) ? v = [n]+v : v << n 
      end
      wc[k] = v
    end
    wc
  end
  #---
  def devide(odd_nums)
    facs = []
    facs[0] = self.map{|key, arr| [key, arr.select{|x| x <= odd_nums[0] or odd_nums[1] < x}] }.to_h
    facs[1] = self.map{|key, arr| [key, arr.select{|x| odd_nums[0] < x and x <= odd_nums[1]}] }.to_h
    facs.map! do |f|
      wc = self.class.new
      wc.each_key{|key| wc[key] = f[key]}
    end
    return facs
  end
  #---
  ## Turaev cobracket for Wcodes
  def cobracket(verbose: false, narrow: true)
    odd_pairs=(1..self.size/2).to_a.map{|x| 2*x-1}.combination(2)
    factors = odd_pairs.map do |odds|
      divs = odds.map{|odd| Division.new(self, odd)}
      linking = divs[0].linking(divs[1])
      factors = self.devide(odds)
      factors.map!(&:narrow) if narrow
      #---
      {odds: odds, linking: linking, factors: factors }
    end
    return verbose ? factors : factors.select{|prd| prd[:linking] != 0}
  end  
end; nil

#-----------------------------------
class Division
  def initialize(wc, odd)
    @wc = wc
    @odd = odd
    wcvf = wc.values.flatten
    @indices = [wcvf.index(@odd), wcvf.index((@odd+1)%(@wc).size)]
    ids = @indices.sort
    @interior = wcvf[ids[0]+1..ids[1]-1]
    @sign = (@indices == @indices.sort) ? 1 : -1
  end
  attr_reader :odd, :indices, :interior, :sign
  
  def linking(another)
    sign1 = @sign
    ao = another.odd
    ae = (ao+1) % @wc.size
    sign2 = if @interior.include?(ao)
               @interior.include?(ae) ? 0 : -1
            else
               @interior.include?(ae) ? 1 : 0
            end
    return sign1*sign2
  end
end;nil

In [6]:
w = comms[1] #Word.new(a[1], a[2], b[1].inverse)
wc = Wcode.new(w)
p w.show, wc.show(3)
printf("===\n")

wcf = wc.values.flatten

divs = (1..wc.size/2).map{|n| 2*n-1}.map{|odd| Division.new(wc, odd)}

divs.each do |div|
  p div.odd, div.interior, div.sign
  printf "---\n"
end

divs.combination(2).each do |comb|
  p "#{comb.map(&:odd)} -- linking --> #{comb[0].linking(comb[1])}"
end
nil

"abAB"
"(5,0|3,6|1,4|7,2)(|||)(|||)"
===
1
[4, 7]
1
---
3
[6, 1]
1
---
5
[0, 3]
1
---
7
[3, 6, 1, 4]
-1
---
"[1, 3] -- linking --> 1"
"[1, 5] -- linking --> 0"
"[1, 7] -- linking --> -1"
"[3, 5] -- linking --> 1"
"[3, 7] -- linking --> 0"
"[5, 7] -- linking --> 1"


In [7]:
wc.cobracket(verbose: 1).each do |t|
#  t.each{ |k,v| puts "#{k}: #{(k == :factors) ? v.map{|f| f.show(2)} : v}" }
  if t[:odds]==[1,3]
    fac = t[:factors][1]
    p fac, fac.show(2)
    printf "---\n"
    fac.each_slice(4){|s| p s,s.size}
  end
end; nil

{"a"=>[], "B"=>[1], "A"=>[], "b"=>[0], "c"=>[], "D"=>[], "C"=>[], "d"=>[], "e"=>[], "F"=>[], "E"=>[], "f"=>[]}
"|1 : |0 : | : | : | : |"
---
[["a", []], ["B", [1]]]
2
[["A", []], ["b", [0]]]
2
[["c", []], ["D", []]]
2
[["C", []], ["d", []]]
2
[["e", []], ["F", []]]
2
[["E", []], ["f", []]]
2


## Turaev cobraket

In [8]:
def cobracket(myw, verbose: false, narrow: true)  
  monomials = Wcode.new(myw).cobracket(verbose: verbose, narrow: narrow).map do |mono|
    prewords = mono[:factors].map(&:narrow).map(&:to_word)
    words = prewords.map(&:cyclic_reduce)
    mono.merge({coeff: mono[:linking], words: words})
  end

  if verbose
    result = monomials
  else
    result = []
    while monomials.size > 0
      pop = monomials.pop
      pop[:odds] = [pop[:odds]]
      #---
      monomials.each do |mono|
        if mono[:words][0].conjugate?(pop[:words][0]) and mono[:words][1].conjugate?(pop[:words][1])
          pop[:coeff] += mono[:coeff]
          pop[:odds] << mono[:odds]
          mono[:coeff] = nil
        elsif mono[:words][0].conjugate?(pop[:words][1]) and mono[:words][1].conjugate?(pop[:words][0])
          pop[:coeff] += mono[:coeff]*(-1)
          pop[:odds] << mono[:odds]
          mono[:coeff] = nil
        end
      end
      #---
      if pop[:coeff] != 0 or verbose
        result << pop
      end
      #---
      monomials.delete_if{|mono| mono[:coeff].nil?}
    end 
  end
  return (result.empty?) ? [{orders: [], linking: 0, factors: [], coeff: 0, words: []}] : result
end; nil

# Execution

In [24]:
a1 = Word.new(a[1])
c = (a1*Word.new(b[1])*(a1.inverse)).flatten

samples = {
  s0: [
    a1, 
    a1*Word.new(b[2]),
    a1*(Word.new(b[2]).inverse),
    a1*c*(c.inverse),
  ],
  s1: [
    a1^4,
    c,
    comms[1],
    Word.new(b[1])*a1*Word.new(b[1])
  ],
  s2: [
    a1*(comms[2].inverse),
    a1*(c.inverse)*(comms[2].inverse)*c,
    a1*(comms[2].inverse)*Group.commutator(comms[2], c.inverse),
    (a1*(comms[2].inverse)*Group.commutator(comms[2], c.inverse)).cyclic_reduce
  ],
  s3: [
    a1*comms[2],
    a1*(comms[2]^2),
    a1*comms[2]*c*comms[2]*(c.inverse),
  ],
  s4: [
    a1*(Word.new(a[2])^3),
    a1*(Word.new(a[2], b[2].inverse)^3)
  ],
  s5: [
    a1*Group.commutator(a[2], a[3]),
    a1*(comms[2]*comms[3])*Group.commutator(a[2], a[3])*((comms[2]*comms[3]).inverse)
  ],
  s6: [
    a1*Group.commutator(b[1], a[2])
  ]
  }

arr = samples #.slice(:s6) #:s2) #, :s3) #:s0, :s1) #
arr.each do |k, v|
  printf "--- #{k} ---\n "
  v.each do |s|
    ws = s.show
    cob = cobracket(s).map do |h|
      coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
      coeff + "#{h[:words].join(" \u{2227} ")}"
    end
    #---
    puts "#{ws}  |--\u{03B4}-->  #{cob.join(' + ')}"
  end
end;nil

--- s0 ---
 a  |--δ-->  (0)
a.d  |--δ-->  (-1)a ∧ d
a.D  |--δ-->  (0)
a.abA.aBA  |--δ-->  ab ∧ B + BA ∧ b + (-1)aB ∧ b + BA ∧ ab + 1 ∧ a
--- s1 ---
 a.a.a.a  |--δ-->  aaa ∧ a + aa ∧ aa + a ∧ aaa
abA  |--δ-->  1 ∧ b
abAB  |--δ-->  (0)
b.a.b  |--δ-->  (0)
--- s2 ---
 a.dcDC  |--δ-->  (-1)a ∧ dcDC
a.aBA.dcDC.abA  |--δ-->  1 ∧ aBAdcDCab
a.dcDC.cdCDaBAdcDCabA  |--δ-->  dcDaBAdcDCab ∧ d + (-1)dcDCDaBAdcDCab ∧ d + (2)1 ∧ aBAdcDCab
aBAdcDCab  |--δ-->  (0)
--- s3 ---
 a.cdCD  |--δ-->  (0)
a.cdCD.cdCD  |--δ-->  acdCD ∧ cdCD
a.cdCD.abA.cdCD.aBA  |--δ-->  (0)
--- s4 ---
 a.c.c.c  |--δ-->  acc ∧ c + ac ∧ cc
a.cD.cD.cD  |--δ-->  acDcD ∧ cD + acD ∧ cDcD
--- s5 ---
 a.ceCE  |--δ-->  (-1)aceC ∧ E + ace ∧ CE + (-1)ac ∧ C
a.cdCD.efEF.ceCE.feFEdcDC  |--δ-->  (-1)acdCDefEFceCfeFEdcDC ∧ E + acdCDefEFcefeFEdcDC ∧ CE + (-1)acdCDefEFcfeFEdcDC ∧ C
--- s6 ---
 a.bcBC  |--δ-->  (-1)abcB ∧ C + abc ∧ BC + (-1)abC ∧ cB


In [20]:
myw = samples[:s6][-1]
wcode = Wcode.new(myw)
p myw.show, wcode.show

vb = true
printf("=====\n")
cobracket(myw, verbose: vb).each do |h|
  h.each do |k,v|
    case k 
    when :odds, :linking
      p "#{k}: #{v}"
    when :factors
      p h[:factors].map(&:show).join(" , ")
    when :words
      coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
      p coeff + "#{h[:words].join(" \u{2227} ")}"
    else
    end
  end
  printf("----------\n")
end; nil

"a.bcBC"
"0|3,6|1|7,2|9,4||5,8|||||"
=====
"odds: [1, 3]"
"linking: 0"
"0|4|1|5|7,2||3,6||||| , |1||0||||||||"
"(0)acBC ∧ b"
----------
"odds: [1, 5]"
"linking: 0"
"0|2|1|3|5||4||||| , |1||0|2||3|||||"
"(0)aBC ∧ bc"
----------
"odds: [1, 7]"
"linking: -1"
"0||1||3||2||||| , |1,4||5,0|2||3|||||"
"(-1)aC ∧ c"
----------
"odds: [1, 9]"
"linking: 0"
"0||1||||||||| , |1,4||5,0|7,2||3,6|||||"
"(0)a ∧ bcBC"
----------
"odds: [3, 5]"
"linking: 1"
"0|3,4|1|5,2|7||6||||| , ||||0||1|||||"
"aC ∧ c"
----------
"odds: [3, 7]"
"linking: -1"
"0|3|1|2|5||4||||| , |2||3|0||1|||||"
"(-1)abC ∧ cB"
----------
"odds: [3, 9]"
"linking: -1"
"0|3|1|2|||||||| , |2||3|5,0||1,4|||||"
"(-1)ab ∧ B"
----------
"odds: [5, 7]"
"linking: 1"
"0|3|1|2|7,4||5,6||||| , |0||1||||||||"
"ab ∧ B"
----------
"odds: [5, 9]"
"linking: 1"
"0|3|1|2|4||5||||| , |0||1|3||2|||||"
"abc ∧ BC"
----------
"odds: [7, 9]"
"linking: -1"
"0|3,6|1|7,2|4||5||||| , ||||1||0|||||"
"(-1)abcB ∧ C"
----------


## Experiments

In [12]:
myw = a1*(comms[2].inverse)
#myw = a1*Group.commutator(b[1], a[2]) #
#myw = a1*Group.conjugate(Group.commutator(b[1], a[2]), c)
wcode = Wcode.new(myw)
p myw.show, wcode.show

vb = true
printf("=====\n")
cobracket(myw, verbose: vb).each do |h|
  h.each do |k,v|
    case k 
    when :odds, :linking
      p "#{k}: #{v}"
    when :factors
      p h[:factors].map(&:show).join(" , ")
    when :words
      coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
      p coeff + "#{h[:words].join(" \u{2227} ")}"
    else
    end
  end
  printf("----------\n")
end; nil

"a.dcDC"
"0||1||9,4|3,6|5,8|7,2||||"
=====
"odds: [1, 3]"
"linking: 0"
"0||1||7,2|4|3,6|5|||| , |||||1||0||||"
"(0)acDC ∧ d"
----------
"odds: [1, 5]"
"linking: 0"
"0||1||5|2|4|3|||| , ||||2|1|3|0||||"
"(0)aDC ∧ dc"
----------
"odds: [1, 7]"
"linking: 0"
"0||1||3||2||||| , ||||2|1,4|3|5,0||||"
"(0)aC ∧ c"
----------
"odds: [1, 9]"
"linking: -1"
"0||1||||||||| , ||||7,2|1,4|3,6|5,0||||"
"(-1)a ∧ dcDC"
----------
"odds: [3, 5]"
"linking: 0"
"0||1||7|3,4|6|5,2|||| , ||||0||1|||||"
"(0)aC ∧ c"
----------
"odds: [3, 7]"
"linking: 0"
"0||1||5|3|4|2|||| , ||||0|2|1|3||||"
"(0)adC ∧ cD"
----------
"odds: [3, 9]"
"linking: 0"
"0||1|||3||2|||| , ||||5,0|2|1,4|3||||"
"(0)ad ∧ D"
----------
"odds: [5, 7]"
"linking: 0"
"0||1||7,4|3|5,6|2|||| , |||||0||1||||"
"(0)ad ∧ D"
----------
"odds: [5, 9]"
"linking: 0"
"0||1||4|3|5|2|||| , ||||3|0|2|1||||"
"(0)adc ∧ DC"
----------
"odds: [7, 9]"
"linking: 0"
"0||1||4|3,6|5|7,2|||| , ||||1||0|||||"
"(0)adcD ∧ C"
----------


In [13]:
p myw.show
d1 = (a1.size)+1
d2 = cif + c.size
d3 = myw.size - (c.size-1)
d4 = myw.size + 1

r1 = (d1..d2-1).map{|n| 2*n-1}.to_a
r2 = (d3..d4-1).map{|n| 2*n-1}.to_a
p r1, r2
p r1.include?(9)
nil

"a.dcDC"


NameError: undefined local variable or method `cif' for main:Object

In [14]:
c = Word.new(a[1], b[1], a[1].inverse)

#myw = a1*(comms[2].inverse) #.cyclic_reduce
#myw = (a1*(c.inverse)*(comms[2].inverse)*c) #.cyclic_reduce
#myw = (a1*c*(comms[2].inverse)*c) #.cyclic_reduce
#myw = a1*Group.commutator(b[1], a[2])
myw = a1*c*(c.inverse)

wcode = Wcode.new(myw)
p myw.show, wcode.show

vb = true #false #
printf("=====\n")

cobra = cobracket(myw, verbose: vb)

=begin
d1 = (a1.size)+1
d2 = d1 + c.size
d3 = myw.size - (c.size-1)
d4 = myw.size + 1
r1 = (d1..d2-1).map{|n| 2*n-1}.to_a
r2 = (d3..d4-1).map{|n| 2*n-1}.to_a
rng = r1+r2
cobra.select! do |h|
  rng.include?(h[:odds][0]) or rng.include?(h[:odds][1])
end
=end

cc = (1..11).to_a.map do |d|
  cobra.select do |h|
    dif = h[:odds][0]-h[:odds][1]
    ((dif).abs/2 == d) and (h[:linking]!=0)
  end
end

#cc.select!{|h| [[1,7],[1,15]].include?(h[:odds])}

cc.flatten.each do |h|
  wstr = myw.show.delete('.')
  idx = h[:odds].map{|n| (n+1)/2}
  str = [wstr[0..idx[0]-1], wstr[idx[0]..idx[1]-1], wstr[idx[1]..-1]].join('|')
  puts "#{h[:odds]}, #{str}"
#  divs = idx.map{ |id| wstr[id-1] + '|' + wstr[id] }
#  puts "#{h[:odds]}, #{str} , #{divs.join(' * ')}"
  h.each do |k,v|
    case k 
    when :linking #, :odds 
#      p "#{k}: #{v}"
    when :factors
#      p h[:factors].map(&:show).join(" , ")
    when :words
      coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
      p coeff + "#{h[:words].join(" \u{2227} ")}"
    else
    end
  end
  printf("----------\n")
end; nil

"a.abA.aBA"
"13,7,0,2,8|5,10|9,3,1,6,12|11,4||||||||"
=====
[1, 3], a|a|bAaBA
"1 ∧ a"
----------
[3, 5], aa|b|AaBA
"BA ∧ b"
----------
[1, 5], a|ab|AaBA
"BA ∧ ab"
----------
[1, 7], a|abA|aBA
"(-1)aB ∧ b"
----------
[7, 13], aabA|aBA|
"ab ∧ B"
----------


### TODO

* [ ] cobracket を再帰的に作用させて、完全に分解することに何らかの意味があるか?
* [ ] $\ell_{2}$ の計算と cobracket との関係は? とくに両者にある division について。
* [ ] a.abA.aBA の cobraket にバグあり。
* [ ] div 同士の linking を div に対応する2文字ワード同士の単純な計算で求められないか?
* [ ] Wcode#devide して得られる2つの Wcode instance の show(2) (および show(3)) がバグってる。なぜか each_slice(4) が2つずつくくる。
* [x] 係数がプラマイ逆かな。
* [x] simple subword たちによる計算を実装する。<-- ダメだった。各 simple subword が内部では確実に交差しない保証がないとダメだから。